### PROGRAMMING FOR AI

### Colour Courier Bot

#### Step 1: Colour Detection Through Images

In [ ]:
import cv2
import numpy as np

# --- Step 1: Read image ---
frame = cv2.imread('black_green(2).jpg')

# --- Step 2: Convert to HSV ---
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# --- Step 3: Define HSV color ranges ---
color_ranges = {
    "Red": [([0, 120, 70], [10, 255, 255]), ([170, 120, 70], [180, 255, 255])],
    "Green": [([36, 100, 100], [86, 255, 255])],
    "Blue": [([94, 80, 2], [126, 255, 255])],
    "Yellow": [([15, 100, 100], [35, 255, 255])],
    "Orange": [([5, 100, 100], [15, 255, 255])]
}

detected_color = None

# --- Step 4: Detect full circle only ---
for color_name, ranges in color_ranges.items():
    mask_total = np.zeros(hsv.shape[:2], dtype=np.uint8)

    for (lower, upper) in ranges:
        lower_np = np.array(lower)
        upper_np = np.array(upper)
        mask = cv2.inRange(hsv, lower_np, upper_np)
        mask_total = cv2.bitwise_or(mask_total, mask)

    # Reduce noise
    mask_total = cv2.morphologyEx(mask_total, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))

    # Find contours
    contours, _ = cv2.findContours(mask_total, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)

        if area > 500 and perimeter > 0:
            circularity = 4 * np.pi * (area / (perimeter * perimeter))

            # Check if shape is circular enough
            if 0.7 < circularity <= 1.2:
                (x, y), radius = cv2.minEnclosingCircle(cnt)
                center = (int(x), int(y))
                radius = int(radius)

                # Check if circle is fully inside the frame
                h, w, _ = frame.shape
                if (center[0] - radius > 0 and center[1] - radius > 0 and
                    center[0] + radius < w and center[1] + radius < h):

                    cv2.circle(frame, center, radius, (255, 255, 255), 2)
                    cv2.putText(frame, color_name, (center[0]-50, center[1]-radius-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                    detected_color = color_name

# --- Step 5: Show result ---
if detected_color:
    print(f"✅ Full circle detected: {detected_color}")
else:
    print("⚠️ No full circle detected")

cv2.imshow("Detected Circle", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


#### Step 2: Colour Detection Through an ESP Camera (AI-THINKER ESP32 CAM)

In [ ]:
import cv2
import numpy as np

url = 'http://192.168.8.100:81/stream'

cap = cv2.VideoCapture(url, cv2.CAP_FFMPEG)

if not cap.isOpened():
    print("Cannot open camera/stream")
    exit()

# ------ COLOR RANGES (HSV) ------
# RED
lower_red1 = np.array([0, 120, 70])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 120, 70])
upper_red2 = np.array([180, 255, 255])

# GREEN
lower_green = np.array([40, 40, 40])
upper_green = np.array([90, 255, 255])

# BLUE
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame")
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # ---- RED mask (two ranges) ----
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = mask_red1 + mask_red2

    # ---- GREEN mask ----
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    # ---- BLUE mask ----
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # Draw bounding boxes
    def detect_color(mask, color_name, frame, color):
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 1000:  # Remove noise
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, color_name, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    detect_color(mask_red, "RED", frame, (0, 0, 255))
    detect_color(mask_green, "GREEN", frame, (0, 255, 0))
    detect_color(mask_blue, "BLUE", frame, (255, 0, 0))

    cv2.imshow("Color Detection", frame)

    if cv2.waitKey(1) == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()

#### Step 3: Testing Through MQTT

In [3]:
import paho.mqtt.client as mqtt
import time

# ===== MQTT CONFIGURATION =====
MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 1883
MQTT_TOPIC = "linefollow/colormatch"
MQTT_CLIENT_ID = "PythonColorDetector_InputVersion"

# Allowed colors
VALID_COLORS = ["red", "blue", "green", "yellow"]

# ===== MQTT SETUP =====
mqtt_client = mqtt.Client(client_id=MQTT_CLIENT_ID)

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print(" Connected to MQTT Broker!")
    else:
        print(f" Failed to connect, return code {rc}")

mqtt_client.on_connect = on_connect

def send_signal_mqtt(match):
    """Send match signal via MQTT"""
    signal = '1' if match else '0'
    try:
        result = mqtt_client.publish(MQTT_TOPIC, signal, qos=1)
        if result.rc == mqtt.MQTT_ERR_SUCCESS:
            print(f" Published: {signal} ({'MATCH' if match else 'NO MATCH'})")
        else:
            print(f" Publish failed: {result.rc}")
    except Exception as e:
        print(f" Error publishing: {e}")

def get_user_color(prompt):
    """Ask the user for a valid color"""
    while True:
        color = input(prompt).strip().lower()
        if color in VALID_COLORS:
            return color
        print("Invalid color! Choose from red, blue, green, yellow.")

def main():
    print("=== Manual Color Match System (MQTT Version) ===")

    # Connect to MQTT Broker
    try:
        print(f"Connecting to broker: {MQTT_BROKER}:{MQTT_PORT}")
        mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)
        mqtt_client.loop_start()
        time.sleep(2)
    except Exception as e:
        print(" MQTT Connection Error:", e)
        return

    print(f" Publishing to topic: {MQTT_TOPIC}")
    print("Type 'q' anytime to quit.\n")

    # Main loop — user input
    while True:
        front = input("Enter FRONT color (red/blue/green/yellow) or 'q' to quit: ").strip().lower()
        if front == "q":
            break

        bottom = input("Enter BOTTOM color (red/blue/green/yellow) or 'q' to quit: ").strip().lower()
        if bottom == "q":
            break

        # Validate and repeat if wrong input
        if front not in VALID_COLORS or bottom not in VALID_COLORS:
            print("Invalid color! Please use red, blue, green, yellow.\n")
            continue

        # Comparison
        match = front == bottom
        print(f"Front: {front} | Bottom: {bottom} | Match: {match}")

        # Send MQTT signal
        send_signal_mqtt(match)

        print()

    mqtt_client.loop_stop()
    mqtt_client.disconnect()
    print("\n=== System Stopped ===")

if __name__ == "__main__":
    main()

C:\Users\user\AppData\Local\Temp\ipykernel_11440\3463452109.py:14: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client(client_id=MQTT_CLIENT_ID)


=== Manual Color Match System (MQTT Version) ===
Connecting to broker: broker.hivemq.com:1883
 Connected to MQTT Broker!
 Publishing to topic: linefollow/colormatch
Type 'q' anytime to quit.

Front: red | Bottom: red | Match: True
 Published: 1 (MATCH)

Front: red | Bottom: red | Match: True
 Published: 1 (MATCH)

Front: yellow | Bottom: yellow | Match: True
 Published: 1 (MATCH)

 Connected to MQTT Broker!

=== System Stopped ===


#### Step 4: Colour Detection and Matching through MQTT & ESP32 CAMS Combined

In [ ]:
import cv2
import numpy as np
import paho.mqtt.client as mqtt
import time

# ===== CAMERA STREAM URLS =====
CAMERA_URL_1 = "http://192.168.8.101:81/stream"   # FRONT CAMERA
CAMERA_URL_2 = "http://192.168.8.102:81/stream"   # BOTTOM CAMERA

# ===== MQTT CONFIGURATION =====
MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 1883
MQTT_TOPIC = "linefollow/colormatch"
MQTT_CLIENT_ID = "PythonColorDetector_CamVersion"

# ===== VALID COLORS =====
VALID_COLORS = ["red", "green", "blue", "yellow", "pink", "orange", "purple"]

# ===== MQTT SETUP WITH ERROR HANDLING =====
mqtt_client = mqtt.Client(client_id=MQTT_CLIENT_ID)
mqtt_connected = False

def on_connect(client, userdata, flags, rc):
    global mqtt_connected
    if rc == 0:
        print("✓ MQTT Connected successfully!")
        mqtt_connected = True
    else:
        print(f"✗ MQTT Connection failed with code: {rc}")
        mqtt_connected = False

def on_disconnect(client, userdata, rc):
    global mqtt_connected
    mqtt_connected = False
    print("✗ MQTT Disconnected!")

mqtt_client.on_connect = on_connect
mqtt_client.on_disconnect = on_disconnect

# Try to connect with error handling
try:
    print("Attempting MQTT connection...")
    mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)
    mqtt_client.loop_start()
    time.sleep(2)  # Wait for connection
    
    if not mqtt_connected:
        print("⚠ WARNING: MQTT not connected. Will work in offline mode.")
except Exception as e:
    print(f"⚠ MQTT Connection Error: {e}")
    print("⚠ Running in OFFLINE mode (no MQTT messages will be sent)")
    mqtt_connected = False

# ===== COLOR DETECTION =====
def detect_color(frame):
    """Detect main color in the center of the frame"""
    if frame is None:
        return None
    
    h, w, _ = frame.shape
    cx, cy = w // 2, h // 2
    
    # Take small region in center
    region = frame[cy-20:cy+20, cx-20:cx+20]
    hsv = cv2.cvtColor(region, cv2.COLOR_BGR2HSV)
    
    # Color ranges in HSV (sensitive version)
    color_ranges = {
        "red": [
            [(0, 70, 40), (10, 255, 255)],
            [(170, 70, 40), (180, 255, 255)]
        ],
        
        "orange": [
            [(11, 80, 80), (25, 255, 255)]
        ],
        
        "yellow": [
            [(26, 80, 80), (35, 255, 255)]
        ],
        
        "green": [
            [(36, 40, 40), (85, 255, 255)]
        ],
        
        "blue": [
            [(86, 80, 40), (130, 255, 255)]
        ],
        
        "purple": [
            [(131, 40, 40), (160, 255, 255)]
        ],
        
        "pink": [
            [(161, 20, 80), (169, 200, 255)],
            [(0, 20, 80), (10, 200, 255)]
        ]
    }
    
    dominant = None
    max_pixels = 0
    
    for color, ranges in color_ranges.items():
        mask = None
        
        for color_range in ranges:
            lower = np.array(color_range[0])
            upper = np.array(color_range[1])
            temp_mask = cv2.inRange(hsv, lower, upper)
            
            if mask is None:
                mask = temp_mask
            else:
                mask = cv2.bitwise_or(mask, temp_mask)
        
        count = cv2.countNonZero(mask)
        if count > max_pixels:
            dominant = color
            max_pixels = count
    
    return dominant

# ===== MQTT SEND WITH ERROR HANDLING =====
def send_match(match):
    if mqtt_connected:
        try:
            mqtt_client.publish(MQTT_TOPIC, "1" if match else "0", qos=1)
            print(f"📤 MQTT SENT: {'MATCH (1)' if match else 'NO MATCH (0)'}")
        except Exception as e:
            print(f"⚠ MQTT Send Error: {e}")
    else:
        print(f"⚠ OFFLINE: Would send {'MATCH (1)' if match else 'NO MATCH (0)'}")

# ===== MAIN LOOP =====
def main():
    print("\n=== Starting Dual Camera Color Detection ===")
    print(f"MQTT Status: {'Connected ✓' if mqtt_connected else 'Offline ✗'}\n")
    
    cap_front = cv2.VideoCapture(CAMERA_URL_1, cv2.CAP_FFMPEG)
    cap_bottom = cv2.VideoCapture(CAMERA_URL_2, cv2.CAP_FFMPEG)
    
    if not cap_front.isOpened():
        print("ERROR: Cannot open front camera stream")
        return
    
    if not cap_bottom.isOpened():
        print("ERROR: Cannot open bottom camera stream")
        return
    
    print("✓ Both cameras connected!")
    
    while True:
        ret1, frame1 = cap_front.read()
        ret2, frame2 = cap_bottom.read()
        
        if not ret1 or not ret2:
            print("⚠ Camera read error!")
            continue
        
        front_color = detect_color(frame1)
        bottom_color = detect_color(frame2)
        
        print(f"Front: {front_color} | Bottom: {bottom_color}")
        
        if front_color and bottom_color:
            match = (front_color == bottom_color)
            send_match(match)
        
        # Draw detection info on frames
        cv2.putText(frame1, f"Front: {front_color}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame2, f"Bottom: {bottom_color}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Show windows for debugging
        cv2.imshow("Front Camera", frame1)
        cv2.imshow("Bottom Camera", frame2)
        
        # Exit by pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap_front.release()
    cap_bottom.release()
    cv2.destroyAllWindows()
    
    if mqtt_connected:
        mqtt_client.loop_stop()
        mqtt_client.disconnect()
        print("\n✓ MQTT Disconnected cleanly")

if __name__ == "__main__":
    main()